In [65]:
import tensorflow as tf
tf.__version__

'1.4.0'

In [66]:
from tensorflow.contrib import keras

In [67]:
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import time
import os

np.random.seed(1337)

In [68]:
from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import sys
import datetime

now = datetime.datetime.now

In [69]:
from matplotlib import pyplot as plt

%matplotlib inline

In [70]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
CLASS_NUM = 21

In [71]:
img_rows, img_cols = IMG_WIDTH, IMG_HEIGHT

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)

In [72]:
from keras.applications import vgg16
from keras.layers import Input

# build the VGG16 network with ImageNet weights
vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input((224, 224, 3)),classes=CLASS_NUM)
print('Model loaded.')

Model loaded.


In [73]:
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

In [74]:
for l in vgg16.layers:
    l.trainable = False

In [75]:
x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
x = Dense(4096, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu', name='ft_fc2')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(CLASS_NUM, activation = 'softmax')(x)

In [76]:
from keras.models import Model

model = Model(inputs=vgg16.input, outputs=predictions)

#compile the model
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True,
        vertical_flip=True)

In [78]:
train_generator = train_datagen.flow_from_directory(
        '../data/train_f2',
        target_size=(224, 224),
        batch_size=32, #changed from 1, change back if needed
        class_mode='categorical')

Found 2615 images belonging to 21 classes.


In [79]:
validation_datagen = ImageDataGenerator()

In [80]:
validation_generator = validation_datagen.flow_from_directory(
        '../data/validation_f2',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 872 images belonging to 21 classes.


In [81]:
test_datagen = ImageDataGenerator()

In [82]:
test_generator = test_datagen.flow_from_directory(
        '../data/test_f2',
        target_size=(224,224),
        batch_size=32,
        class_mode='categorical')

Found 388 images belonging to 21 classes.


In [83]:
from keras.preprocessing import image
from keras.callbacks import TensorBoard

In [84]:
model.fit_generator(
        train_generator,
        steps_per_epoch=82,
        epochs=40,
        validation_data=validation_generator,
        validation_steps=28,
        use_multiprocessing=True)
#         callbacks=[TensorBoard(log_dir='./tf_logs', histogram_freq=0, 
#                                write_graph=True, write_images=True, 
#                                embeddings_freq=10, 
#                                embeddings_layer_names=['block1_conv2', 
#                                                        'block5_conv1', 
#                                                        'ft_fc1'], 
#                                embeddings_metadata=None)])

Epoch 1/40
82/82 [==============================] - 95s 1s/step - loss: 3.3782 - acc: 0.1056 - val_loss: 2.9659 - val_acc: 0.2294
Epoch 2/40
82/82 [==============================] - 86s 1s/step - loss: 2.9437 - acc: 0.1637 - val_loss: 2.5064 - val_acc: 0.2477
Epoch 3/40
82/82 [==============================] - 85s 1s/step - loss: 2.7324 - acc: 0.2203 - val_loss: 2.3113 - val_acc: 0.2901
Epoch 4/40
82/82 [==============================] - 86s 1s/step - loss: 2.6238 - acc: 0.2397 - val_loss: 2.1691 - val_acc: 0.3417
Epoch 5/40
82/82 [==============================] - 86s 1s/step - loss: 2.4943 - acc: 0.2577 - val_loss: 2.0781 - val_acc: 0.3486
Epoch 6/40
82/82 [==============================] - 87s 1s/step - loss: 2.4221 - acc: 0.2709 - val_loss: 2.0833 - val_acc: 0.3532
Epoch 7/40
82/82 [==============================] - 87s 1s/step - loss: 2.2823 - acc: 0.2979 - val_loss: 2.0572 - val_acc: 0.3658
Epoch 8/40
82/82 [==============================] - 86s 1s/step - loss: 2.2494 - acc: 0.30

In [85]:
model_history = _

In [86]:
model.evaluate_generator(validation_generator,steps=15)

[1.5314044252448125, 0.5240825688073395]

In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [32]:
model.save_weights('weight_f1.h5')

In [87]:
import cv2

In [91]:
import pandas as pd
df = pd.read_csv('../data/test_f2/xy.txt')
df.head()

,file_name,order
0,picture2191.jpg,Caenidae
1,photo_1319939.jpg,Ephemerellidae
2,picture4569.jpg,Baetidae
3,picture2394.jpg,Ephemeridae
4,photo_1397136.jpg,Ephemeridae


In [92]:
t = test_generator.class_indices

In [94]:
y_cat = np_utils.to_categorical([t[i] for i in df.order], 21)
y_cat

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [95]:
iml = []
for f,o in zip(df.file_name,df.order):
    i_path = '../data/test_f2/{}/{}'.format(o,f)
    iml.append(cv2.resize(cv2.imread(i_path,1),(224,224),interpolation = cv2.INTER_AREA))

In [96]:
y_cat = np.array(y_cat)
y_true = []
y_pred = []


iml = np.stack(iml)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(iml,y_cat,
        batch_size=32)

In [97]:
pred = model.predict(iml)

In [98]:
inv_map = {v: k for k, v in t.items()}
y_pred = [inv_map[np.argmax(i)] for i in pred]

In [99]:
y_true = [inv_map[np.argmax(i)] for i in y_cat]

In [100]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred))

                  precision    recall  f1-score   support

        Baetidae       0.35      0.21      0.27        28
     Baetiscidae       0.62      0.57      0.59        14
        Caenidae       0.11      0.33      0.16         6
       Capniidae       1.00      0.20      0.33         5
  Chloroperlidae       0.33      0.29      0.31         7
  Ephemerellidae       0.62      0.47      0.53        88
     Ephemeridae       0.68      0.54      0.60        28
   Heptageniidae       0.75      0.17      0.28        71
  Hydropsychidae       0.08      0.09      0.09        11
    Isonychiidae       1.00      0.33      0.50         6
    Leptoceridae       0.33      0.38      0.35         8
   Leptohyphidae       0.22      0.52      0.31        21
   Limnephilidae       0.50      0.40      0.44        20
          Midges       0.67      0.67      0.67         3
      Nemouridae       0.24      0.82      0.37        11
        Perlidae       0.54      0.56      0.55        25
      Perlodi

In [121]:
a = np.unique(y_true, return_counts =True)

y_true2 = [i if i in a[0][a[1]>15] else 'other' for i in y_true] 
y_pred2 = [i if i in a[0][a[1]>15] else 'other' for i in y_pred]
# for i,j in zip(y_true,y_pred):
#     if i in a[0][a[1]>15]:
#         if j not in a[0][a[1]>15]:
#             y_true2.append(i)
#             y_pred2.append('other')
#         y_true2.append(i)
#         y_pred2.append(j)
#     else:
#         y_true2.append('other')
#         y_pred2.append('other')


388

In [118]:
a[0][a[1]>15]

array(['Baetidae', 'Ephemerellidae', 'Ephemeridae', 'Heptageniidae',
       'Leptohyphidae', 'Limnephilidae', 'Perlidae', 'Perlodidae'],
      dtype='<U16')

In [124]:
print(classification_report(y_true2,y_pred2))

                precision    recall  f1-score   support

      Baetidae       0.35      0.21      0.27        28
Ephemerellidae       0.62      0.47      0.53        88
   Ephemeridae       0.68      0.54      0.60        28
 Heptageniidae       0.75      0.17      0.28        71
 Leptohyphidae       0.22      0.52      0.31        21
 Limnephilidae       0.50      0.40      0.44        20
      Perlidae       0.54      0.56      0.55        25
    Perlodidae       0.35      0.50      0.41        16
         other       0.43      0.71      0.54        91

   avg / total       0.54      0.46      0.45       388

